# Initialisation

In [226]:
import matplotlib
matplotlib.use('Qt5Agg')  # Utiliser le backend Qt5Agg pour Windows
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import cv2
import numpy as np
import os
from PIL import Image
import lmfit
#import tifffile
#%matplotlib tk #pour Linux

In [227]:
import scipy.ndimage as ndimage
from sklearn.cluster import DBSCAN

# Importer fichier

In [228]:
path_to_tiff = os.path.join("..", "acquisition", "video_output_carac_150ms_1im_10um.tiff")

tiff = Image.open(path_to_tiff)

# Nombre de frames pas vide

In [229]:
with Image.open(path_to_tiff) as img:
    frame_number = 0
    actual_frames = 0
    try:
        while True:
            frame_number += 1
            if np.sum(np.array(img)) != 0:
                actual_frames += 1
                
            img.seek(frame_number)
    except EOFError:
        print("All frames processed.")

actual_frames

All frames processed.


62

# 1re frame

**Le nombre de frames ignorés n'est pas pris en compte**

In [230]:
frame_index = 0
first_frame = 0
tiff.seek(frame_index)
original_image = np.array(tiff)

while np.sum(original_image) == 0:
    frame_index += 1
    tiff.seek(frame_index)
    original_image = np.array(tiff)
    first_frame += 1
    print(frame_index)




# Traitement d'image

In [231]:
clahe = cv2.createCLAHE(clipLimit=10.0, tileGridSize=(30, 30))
preprocessed = clahe.apply(original_image)

blurred = cv2.medianBlur(preprocessed, 115)
preprocessed2 = cv2.subtract(preprocessed, blurred)

# Apply Non-Local Means Denoising
img = cv2.fastNlMeansDenoising(preprocessed2, None, 15, 7, 41)
print(img)

[[10 12 12 ...  9 10  9]
 [11 12 11 ...  9  9  9]
 [11 12 11 ...  9  9  9]
 ...
 [ 7  7  7 ...  6  6  7]
 [ 7  7  7 ...  6  6  6]
 [ 7  8  7 ...  6  6  6]]


# Sélection du point à tracker

In [232]:
def crop(img, x, y, crop_size_x, crop_size_y):
    x_start = int(x - crop_size_x // 2)
    x_end = int(x + crop_size_x // 2)
    y_start = int(y - crop_size_y // 2)
    y_end = int(y + crop_size_y // 2)

    return img[y_start:y_end, x_start:x_end]

In [233]:
# Display the image and let the user select a point interactively
fig, ax = plt.subplots()
ax.imshow(img, origin='lower', cmap='gray')  # Use 'gray' for better visibility of grayscale images
plt.title(f"Frame {frame_index}: Select a point")

# Ask for a point to be selected
print("Please click on the point you want to select.")
x, y = plt.ginput(1)[0]  # This will get the coordinates of the clicked point
print(f"Selected point: ({x}, {y})")
plt.close()

crop_sze_x = 250
crop_sze_y = 400

Please click on the point you want to select.
Selected point: (351.31818181818187, 321.51298701298697)


Debogueur

In [234]:
def visionneur(frame, titre):
    plt.figure(figsize=(10, 5))
    plt.clf() 
    plt.imshow(frame, origin='lower', cmap='gray')
    plt.title(titre)
    plt.colorbar()  
    plt.show()

# Fit gaussien sur le point sélectionné

In [235]:
def prepare_data(x, y, z):
    return (x.flatten(), y.flatten()), z.flatten()

In [236]:
def gaussian_2d(xy, amplitude, x0, y0, sigma_x, sigma_y, offset):
    x, y = xy
    a = 1 / (2 * sigma_x**2)
    b = 1 / (2 * sigma_y**2)
    return offset + amplitude * np.exp(- (a * (x - x0)**2 + b * (y - y0)**2))

In [237]:
def localisateur_gaussien(intensity_grid, centre):
    x = np.arange(intensity_grid.shape[0])
    y = np.arange(intensity_grid.shape[1])
    X, Y = np.meshgrid(x, y)

    # Préparer les données pour le fit
    (xdata, ydata), zdata = prepare_data(X, Y, intensity_grid)
    model = lmfit.Model(gaussian_2d)
    max_idx = np.unravel_index(np.argmax(intensity_grid), intensity_grid.shape)
    initial_x0 = x[max_idx[0]]
    initial_y0 = y[max_idx[1]]

    # Définir les paramètres du modèle
    params = model.make_params(
        amplitude=np.max(intensity_grid),
        x0=initial_x0,
        y0=initial_y0,
        sigma_x=1,
        sigma_y=1,
        offset=2
    )

    # Effectuer l'ajustement
    result = model.fit(zdata, params, xy=(xdata, ydata))
    
    x_position = result.params['x0'].value + centre[0] - crop_sze_x/2
    y_position = result.params['y0'].value + centre[1] - crop_sze_y/2

    return [x_position, y_position], result.params['sigma_x'].value, result.params['sigma_y'].value

# Process d'image (enlever le bruit)

**semble faire du trouble**

In [238]:
def denoise(image):
    clahe = cv2.createCLAHE(clipLimit=10.0, tileGridSize=(30, 30))
    preprocessed = clahe.apply(image)
    
    blurred = cv2.medianBlur(preprocessed, 115)
    preprocessed2 = cv2.subtract(preprocessed, blurred)
    
    return cv2.fastNlMeansDenoising(preprocessed2, None, 15, 7, 41)

# Identifier la particule et donner sa nouvelle position

In [239]:
def empreinte_digitale(image, threshold=50, eps=3, min_samples=1):
    # 1. Observer chaque particule approximativement
    #neighborhood_size = 3  
    #local_max = ndimage.maximum_filter(image, size=neighborhood_size)
    visionneur(image, f'particule choisie')

    maxima = (image > threshold)
    
    coordinates = np.column_stack(np.where(maxima))  
    if coordinates.shape[0] == 0:
        print("Aucune particule détectée.")
        return [], image

    dbscan = DBSCAN(eps=eps, min_samples=min_samples)
    labels = dbscan.fit_predict(coordinates)
    unique_labels = np.unique(labels)
    size=0

    # 2. Déterminer l'empreinte digitale de notre particule
    for label in unique_labels:
        if label != -1:  # -1 correspond au bruit dans DBSCAN
            cluster_points = coordinates[labels == label]
            plt.scatter(cluster_points[:, 1], cluster_points[:, 0], label=f'Particule {label}')

            if cluster_points.shape[0]>size:
                size = cluster_points.shape[0]
                luminosity = np.mean(image[cluster_points[:, 0], cluster_points[:, 1]])

    seuil=np.max(image)*0.75
    print(seuil)
    # 3. Afficher les résultats et filtrer les particules selon leur taille et luminosité
    plt.imshow(image, origin='lower', cmap='gray')
    plt.title(f"Empreinte digitale de la particule, lum={luminosity} et taille={size}")
    plt.legend()
    plt.colorbar()  
    plt.show()

    return size, luminosity, seuil

In [240]:
def detect_nbre_particles(image, threshold, eps=3, min_samples=1):
    # 1. Appliquer un filtre de voisinage pour détecter les maxima locaux
    neighborhood_size = 3  # Taille du voisinage pour détecter les maxima locaux
    local_max = ndimage.maximum_filter(image, size=neighborhood_size)
    # 2. Comparer l'image originale et les maxima locaux pour identifier les vrais maxima
    maxima = (image == local_max) & (image > threshold)

    # 3. Extraire les coordonnées des maxima (particules)
    coordinates = np.column_stack(np.where(maxima))  # Extraire les indices des pixels maximaux
    if coordinates.shape[0] == 0:
        print("Aucune particule détectée.")
        visionneur(image, f'détection du nombre de particules')

        return [], image

    # 4. Appliquer DBSCAN pour regrouper les particules proches (cluster les maxima détectés)
    dbscan = DBSCAN(eps=eps, min_samples=min_samples)
    labels = dbscan.fit_predict(coordinates)

    #5. Déterminer le nombre de particules
    unique_labels = np.unique(labels)
    nb_particules = len(unique_labels)
        
    # 6. Afficher les résultats et filtrer les particules selon leur taille et luminosité
    plt.imshow(image, origin='lower', cmap='gray')
    plt.title("Détection des particules")
    
    positions=[]
    for label in unique_labels:
        if label != -1:  # -1 correspond au bruit dans DBSCAN
            cluster_points = coordinates[labels == label]
            plt.scatter(cluster_points[:, 1], cluster_points[:, 0], label=f'Particule {label}')
#        if nb_particules > 1:
        positions.append((np.mean(cluster_points[:, 1]), np.mean(cluster_points[:, 0])))
        

    plt.colorbar()  
    plt.legend()
    plt.show()

    # Retourner les coordonnées des particules filtrées
    return nb_particules, positions

In [241]:
def identificateur_particles(image, ref_size, ref_luminosity, threshold=25, eps=3, min_samples=1):
    # 1. Obtenir le plus d'info sur la particule
    maxima = (image > threshold)
    
    coordinates = np.column_stack(np.where(maxima))  
    if coordinates.shape[0] == 0:
        print("Mauvais re-crop empêche identification de la particule.")
        return [], image

    dbscan = DBSCAN(eps=eps, min_samples=min_samples)
    labels = dbscan.fit_predict(coordinates)
    unique_labels = np.unique(labels)

    # 2. Parcourir chaque particule et identifier laquelle est celle qu'on suit
    distances = []
    position_particules = []

    for label in unique_labels:
        if label != -1:  # -1 correspond au bruit dans DBSCAN
            cluster_points = coordinates[labels == label]
            
            size = cluster_points.shape[0]
            luminosity = np.mean(image[cluster_points[:, 0], cluster_points[:, 1]])
            size_distance = abs(size - ref_size)  
            luminosity_distance = abs(luminosity - ref_luminosity)  
            combined_distance = size_distance + luminosity_distance
            distances.append(combined_distance)
            position_particules.append((np.mean(cluster_points[:, 1]), np.mean(cluster_points[:, 0])))

    ecart = np.min(distances)
    position_best_particle = position_particules[np.argmin(distances)]  
    print(f'ecrat: {ecart}')
    # 3. Afficher les résultats et filtrer les particules selon leur taille et luminosité
    plt.imshow(image, origin='lower', cmap='gray')
    plt.title("Détection des particules")
    plt.scatter(position_best_particle[0], position_best_particle[1], color='r', label=f'Particule sélectionnée')
    plt.legend()
    plt.colorbar()  
    plt.show()

    return position_best_particle, ecart

# Faire le crop et fit

In [242]:
def particle_tracker_simple(image, x, y):
    image = denoise(image)

    cropped_img = crop(image, x, y, crop_sze_x, crop_sze_y)
    
    #Gérer plus qu'une particule
    cropped_img = np.array(cropped_img)
    max_index = np.argmax(cropped_img)
    max_coords = np.unravel_index(max_index, cropped_img.shape)

    nouveau_x = x - crop_sze_x // 2 + max_coords[1]
    nouveau_y = y - crop_sze_y // 2 + max_coords[0]
    
    second_crop = crop(image, nouveau_x, nouveau_y, crop_sze_x, crop_sze_y)    # Re-crop autour d'une seule particule

    result_fit = localisateur_gaussien(second_crop, [nouveau_x, nouveau_y])

    x_new, y_new = result_fit[0][0], result_fit[0][1]

    return [result_fit, cropped_img, (x_new, y_new), (result_fit[1], result_fit[2])]

In [243]:
#Placés ici pour pouvoir y faire référence dans particule_tracker
position_list=[]
sigma_list = []
crop_frames = []
big_frames = []
position_list.append([x,y])

In [244]:
def particle_tracker(image, x, y, taille_initiale, luminosité_initiale, seuil):
    # 1. Ce qu'on voit près de l'ancienne position
    image = denoise(image)

    cropped_img = crop(image, x, y, crop_sze_x, crop_sze_y)
    
    # 2. Déterminer combien il y a de particules et laquelle est la notre
    # Utiliser une reconnaissance de particules et si plus qu'une, alors clic pour choisir
    nb_part, coordonnées = detect_nbre_particles(cropped_img, seuil)
    print(f'coordonnées des {nb_part} particules:{coordonnées}')

    if nb_part == 1:
        cropped_img = np.array(cropped_img)
        max_index = np.argmax(cropped_img)
        max_coords = np.unravel_index(max_index, cropped_img.shape)
    #print(F'coordonnées du max dans le premier crop: ({max_coords[1]},{max_coords[0]})')

        nouveau_x = x - crop_sze_x // 2 + max_coords[1] #Oui c'est inversé à cause du unravel juste avant
        nouveau_y = y - crop_sze_y // 2 + max_coords[0]
    else:
        # Display the image and let the user select a point interactively
        fig, ax = plt.subplots()
        ax.imshow(denoise(img), origin='lower', cmap='gray') 
        plt.title(f"Frame {len(position_list)-1}: Select a point")

        print(len(position_list)-1)
        # Ajouter une croix rouge à la position donnée
        cx, cy = position_list[len(position_list)-1]
        ax.plot(cx, cy, 'rx', markersize=10) 
        
        # Afficher l'image et demander à l'utilisateur de cliquer
        print("Please click on the point you want to select.")
        nouveau_x, nouveau_y = plt.ginput(1)[0]  # Attente du clic de l'utilisateur (1 point)
        print(f"Selected point: ({nouveau_x}, {nouveau_y})")
        
        # Fermer l'affichage après sélection
        plt.close()
    

    #print(f'(x,y) central du second crop: {nouveau_x, nouveau_y}')
    
    # 3. Crop atour de notre particule et fit dessus
    second_crop = crop(image, nouveau_x, nouveau_y, crop_sze_x//2, crop_sze_y//2)    # Re-crop autour d'une seule particule


    result_fit = localisateur_gaussien(second_crop, [nouveau_x, nouveau_y])
    #print(f'écart-type: ({result_fit[1]}, {result_fit[2]})')

    x_new, y_new = result_fit[0][0], result_fit[0][1]
    print(x_new, y_new)
    #Affiche l'accord entre le fit gaussien et le blob (pour comparer)
    x_range = np.arange(crop_sze_x) - crop_sze_x//2 
    y_range = np.arange(crop_sze_y) - crop_sze_y//2
    x_position = x_range + x_new  
    y_position = y_range + y_new 
    plt.imshow(second_crop, origin='lower', cmap='gray', extent=[x_position.min(), x_position.max(), y_position.min(), y_position.max()])
    plt.title("positionnement de la particule")
    plt.scatter(x_new, y_new, color='r', label=f'Position estimée par fit gaussien')
    plt.scatter(nouveau_x - crop_sze_x//4, nouveau_y - crop_sze_y//4, color='r', marker='+', label=f'Position meilleure particule')
    plt.legend()
    plt.colorbar()  
    plt.show()

    return [result_fit, second_crop, (x_new, y_new), (result_fit[1], result_fit[2])]

# Main loop

In [245]:
taille_initiale, luminosité_initiale, seuil = empreinte_digitale(crop(img,x,y,crop_sze_x*0.8,crop_sze_y*0.8))


for i in range(first_frame,17):
    tiff.seek(i)
    if np.sum(np.array(tiff))==0:
        position_list.append(np.array([np.nan,np.nan]))
        sigma_list.append(np.array([np.nan,np.nan]))
        print("hello")
    else:
        data = particle_tracker(img, x, y, taille_initiale, luminosité_initiale, seuil)
        position_list.append(data[2])
        x,y = position_list[i-first_frame]
        sigma_list.append(data[3])
        tiff.seek(i)
        img = np.array(tiff)
        big_frames.append(img)
        crop_frames.append(data[1])
    print(position_list)


108.75
coordonnées des 2 particules:[(122.85185185185185, 202.5185185185185), (114.0, 399.0)]
0
Please click on the point you want to select.
Selected point: (342.5519480519481, 327.3571428571429)
266.8305655625892 188.16507688452884
[[351.31818181818187, 321.51298701298697], (266.8305655625892, 188.16507688452884)]
coordonnées des 2 particules:[(121.84210526315789, 201.8736842105263), (113.2, 399.0)]
1
Please click on the point you want to select.
Selected point: (342.5519480519481, 324.43506493506493)
-128.88045758397652 185.82751713227447
[[351.31818181818187, 321.51298701298697], (266.8305655625892, 188.16507688452884), (-128.88045758397652, 185.82751713227447)]
Aucune particule détectée.


C:\Users\maxim\AppData\Local\Temp\ipykernel_24592\629413214.py:4: UserWarning: Attempting to set identical low and high ylims makes transformation singular; automatically expanding.
  plt.imshow(frame, origin='lower', cmap='gray')


coordonnées des [] particules:[]
2
Please click on the point you want to select.
Selected point: (350.48744033886794, 278.5354049104726)
346.3699421411432 149.64761321752707
[[351.31818181818187, 321.51298701298697], (266.8305655625892, 188.16507688452884), (-128.88045758397652, 185.82751713227447), (346.3699421411432, 149.64761321752707)]
Aucune particule détectée.
coordonnées des [] particules:[]
3
Please click on the point you want to select.
Selected point: (348.3961038961039, 280.6038961038961)
307.99835676728276 145.60313050162193
[[351.31818181818187, 321.51298701298697], (266.8305655625892, 188.16507688452884), (-128.88045758397652, 185.82751713227447), (346.3699421411432, 149.64761321752707), (307.99835676728276, 145.60313050162193)]
Aucune particule détectée.
coordonnées des [] particules:[]
4
Please click on the point you want to select.
Selected point: (357.1623376623377, 251.3831168831169)
345.7278868082574 109.38335987670769
[[351.31818181818187, 321.51298701298697], (266

ValueError: attempt to get argmax of an empty sequence

In [ ]:
pixel_size = 3.45
sigma_arr=np.array(sigma_list)* pixel_size * 10**(-6)
position_arr=np.array(position_list)* pixel_size * 10**(-6)
print(position_arr)

[[ 0.00110433  0.00323368]
 [ 0.00393632  0.00322736]
 [ 0.00448275  0.00151883]
 [ 0.00108805  0.00145414]
 [ 0.00179745  0.00133555]
 [ 0.00094635  0.00126199]
 [ 0.00113038  0.00118603]
 [ 0.00142269  0.0010817 ]
 [ 0.00507411  0.0009989 ]
 [-0.00035422  0.00090838]
 [-0.00052588  0.00103452]
 [-0.00082439  0.00124621]
 [-0.00077118  0.00132122]
 [-0.00073339  0.00120298]
 [-0.00079339  0.00140971]
 [-0.00094828  0.00137468]
 [-0.00077041  0.00151165]]


In [ ]:
def calculate_msd_with_uncertainty(position_arr, delta_x, delta_y):
    """
    Calcule les MSD avec propagation des incertitudes analytiques.
    """


    msd = []
    uncertainties = []
    for d in range(1, len(position_arr)):
        diff_pairs = np.zeros_like(position_arr[d:])
        dx = np.zeros_like(delta_x[d:])
        dy = np.zeros_like(delta_y[d:])

        # Masque pour détecter les NaN
        nan_mask_pos = np.isnan(position_arr[d:]) | np.isnan(position_arr[:-d])
        nan_mask_delta = np.isnan(delta_x[d:]) | np.isnan(delta_x[:-d])
        
        # Calculer les différences uniquement où nan_mask est False
        valid_mask_pos = ~nan_mask_pos.any(axis=1)  # Inverser le masque pour obtenir les position_arr valides
        valid_mask_delta = ~nan_mask_delta
        
        diff_pairs[valid_mask_pos] = position_arr[d:][valid_mask_pos, :] - position_arr[:-d][valid_mask_pos, :]

        distances_squared = np.sum(diff_pairs**2, axis=1)
        msd.append(np.mean(distances_squared))  
        
        dx[valid_mask_delta] = delta_x[d:][valid_mask_delta] + delta_x[:-d][valid_mask_delta]
        dy[valid_mask_delta] = delta_y[d:][valid_mask_delta] + delta_y[:-d][valid_mask_delta]
        term_x = 2 * (diff_pairs[:, 0]**2) * (dx**2)
        term_y = 2 * (diff_pairs[:, 1]**2) * (dy**2)
        
        # Propagation des incertitudes
        total_uncertainty = np.mean(term_x + term_y)
        uncertainties.append(np.sqrt(total_uncertainty))

    return np.array(msd), np.array(uncertainties)

calculate_msd_with_uncertainty(position_arr, sigma_arr[:, 0], sigma_arr[:, 1])

(array([4.20556332e-06, 5.62391805e-06, 5.91336351e-06, 6.44438671e-06,
        7.65923834e-06, 6.40816749e-06, 9.28183237e-06, 1.40960187e-05,
        1.03488529e-05, 1.13065824e-05, 1.24772945e-05, 1.43413633e-05,
        1.65372579e-05, 2.06052367e-05, 1.63830358e-05, 6.48002145e-06]),
 array([2.97838603e-04, 5.94230233e-04, 4.75187082e-04, 7.46323655e-04,
        5.49348910e-04, 5.81647922e-04, 6.63116923e-04, 6.87803443e-04,
        7.14589857e-04, 7.62318206e-04, 8.73989202e-04, 8.94305420e-04,
        1.37638305e-04, 1.50853104e-04, 5.04264129e-05, 1.39238644e-05]))

# Résultats

In [ ]:
# Assuming msd_values is the result of your MSD calculation
msd_values, uncertainties = calculate_msd_with_uncertainty(position_arr, sigma_arr[:, 0], sigma_arr[:, 1])
print('coucou')
# Time intervals for the x-axis (assuming a uniform time step)
time_intervals = np.arange(1, len(msd_values) + 1) * 0.5

# Plotting the MSD
plt.figure(figsize=(8, 6))
plt.errorbar(time_intervals, msd_values, yerr=uncertainties, fmt='o', label='MSD', color='blue')

# Customize the plot
plt.title("MSD en fonction de l'intervalle de temps")
plt.xlabel("Intervalle de temps")
plt.ylabel("MSD")
plt.grid(True)
plt.legend()

# Show the plot
plt.show()

coucou


In [ ]:
from scipy.optimize import curve_fit

# Fonction quadratique pour l'ajustement
def quadratic(x, a, b, c):
    return a * x**2 + b * x + c

# Assuming msd_values is the result of your MSD calculation
msd_values, uncertainties = calculate_msd_with_uncertainty(position_arr, sigma_arr[:, 0], sigma_arr[:, 1])

cropped_msd = msd_values[:5]
cropped_inc = uncertainties[:5]

# Time intervals for the x-axis (assuming a uniform time step)
time_intervals = np.arange(1, len(cropped_msd) + 1)

# Perform a quadratic fit using curve_fit, which gives both coefficients and covariance matrix
popt, pcov = curve_fit(quadratic, time_intervals, cropped_msd, sigma=cropped_inc)

# popt contains the optimal coefficients
coeffs = popt

# pcov is the covariance matrix, which we can use to compute uncertainties
coeff_uncertainties = np.sqrt(np.diag(pcov))  # Incertitudes sur les coefficients sont les éléments diagonaux

# Affichage des coefficients séparément
print("Coefficients du polynôme ajusté :")
print(coeffs)

# Affichage de la matrice de covariance séparément
print("Matrice de covariance des coefficients :")
print(pcov)

# Création de la fonction quadratique de l'ajustement
quadratic_fit = np.poly1d(coeffs)

# Génération des valeurs ajustées pour afficher la courbe
fitted_msd = quadratic_fit(time_intervals)

# Affichage du MSD avec les barres d'erreur
plt.figure(figsize=(8, 6))
plt.errorbar(time_intervals, cropped_msd, yerr=cropped_inc, fmt='o', label='MSD', color='blue')
plt.plot(time_intervals, fitted_msd, label='Quadratic Fit', color='red', linestyle='--')

# Personnalisation du graphique
plt.title("MSD en fonction de l'intervalle de temps")
plt.xlabel("Intervalle de temps")
plt.ylabel("MSD")
plt.grid(True)
plt.legend()

# Afficher le graphique
plt.show()

# Affichage des incertitudes sur les coefficients
print(f"Incertitudes sur les coefficients : {coeff_uncertainties}")

# Calcul du coefficient de diffusion et de la taille de la particule
r = (4 * 1.38 * 10**-23 * 300 / (6 * np.pi * 10**(-3) * coeffs[1]))  # Coefficient de diffusion

# Affichage de la taille de la particule avec incertitude
print(f"Taille de la particule (m): {r:.2e} m, avec une incertitude : {coeff_uncertainties[1]:.2e}")


Coefficients du polynôme ajusté :
[-3.10938861e-08  1.00676039e-06  3.28000834e-06]
Matrice de covariance des coefficients :
[[ 7.60173226e-15 -4.26857938e-14  4.14914025e-14]
 [-4.26857938e-14  2.49080641e-13 -2.54417564e-13]
 [ 4.14914025e-14 -2.54417564e-13  2.95570401e-13]]
Incertitudes sur les coefficients : [8.71879135e-08 4.99079794e-07 5.43663868e-07]
Taille de la particule (m): 8.73e-13 m, avec une incertitude : 4.99e-07


In [ ]:
x_plt, y_plt = zip(*position_list)
# Create the plot
plt.figure(figsize=(10, 8))  # Optional: Adjust the figure size
plt.plot(x_plt, y_plt, marker='o', linestyle='-', color='b', label='Connected Points')

# Inverser l'axe Y
plt.gca().invert_yaxis()

# Set grid limits to match the 1440x1080 grid
plt.xlim(0, 1440)
plt.ylim(0, 1080)

# Add labels and title
plt.xlabel('X-axis')
plt.ylabel('Y-axis')
plt.title('2D Connected Points Plot')
plt.legend()

# Optional: Add grid lines
plt.grid(True)

# Afficher le graphique
plt.show()

## anim toutes les particules

In [ ]:
fig, ax = plt.subplots()
img = ax.imshow(big_frames[0], origin='lower', cmap='gray', animated=True)


# Update function
def update(frame):
    img.set_array(frame)
    return img,
    
ani = animation.FuncAnimation(fig, update, frames=big_frames, interval=50, blit=True)
plt.show()

## anim crop

In [ ]:
fig, ax = plt.subplots()
img = ax.imshow(crop_frames[0], origin='lower', cmap='gray', animated=True)


def update(frame):
    img.set_array(frame)
    return img,
    
ani = animation.FuncAnimation(fig, update, frames=crop_frames, interval=50, blit=True)
plt.show()